# first attempt - simple reverse prediction

In [1]:
import numpy as np
import pandas as pd
from functions import *
from read_from_file_df import *
import pickle
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [2]:
SIZE = 5
AMOUNT_BOARDS = 100000
AMOUNT_MOVES = 100
NUM_DICT = 1

In [3]:
gen = 2
name_df = f'{PATH_DF}\\{SIZE}-{AMOUNT_BOARDS}\\{SIZE}size_{AMOUNT_BOARDS}boards_{gen}gen_reverse'
reverse_df = pd.read_pickle(f'{name_df}.pkl')

In [4]:
print(name_df)

C:\GameOfLifeFiles\df\\5-100000\5size_100000boards_2gen_reverse


In [5]:
new_columns = [f'Col_{i}' for i in range(1, gen*SIZE*SIZE + 1)]
reverse_df_sort = reverse_df.sort_values(by = new_columns).reset_index(drop=True)
for i in reverse_df_sort.columns:
    reverse_df_sort[i] = reverse_df_sort[i].astype(int)

In [6]:
print("reverse df:", len(reverse_df))
print("reverse df sort:",len(reverse_df_sort))

reverse df: 162937
reverse df sort: 162937


In [7]:
# Step 1: Prepare Data
amount_features = len(reverse_df_sort.columns) - SIZE*SIZE #the previous boards
features = reverse_df_sort.iloc[:, :amount_features]
name_col = 'Col_' + str(amount_features + 1)  # Target: the first pixel in the board
target = reverse_df_sort[name_col]

# Step 2: Split Data
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.1, random_state=365)

print("len x train: ", len(X_train))
print("len x test: ",len(X_test))
print("len y train: ",len(y_train))
print("len y test: ",len(y_test))

dt = tree.DecisionTreeClassifier(random_state=42)
dt = dt.fit(X_train, y_train)

y_train_pred = dt.predict(X_train)
y_test_pred = dt.predict(X_test)

train_test_full_error = pd.concat([measure_error(y_train, y_train_pred, 'train'),
                                measure_error(y_test, y_test_pred, 'test')],
                                axis=1)

print(dt.tree_.node_count, dt.tree_.max_depth)
train_test_full_error

len x train:  146643
len x test:  16294
len y train:  146643
len y test:  16294
103035 25


,train,test
accuracy,0.957107,0.628882
precision,0.985564,0.495369
recall,0.894520,0.463887
f1,0.937837,0.479111


In [8]:
reverse_df_sort.iloc[17].values

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 1, 0])

In [9]:
features.iloc[17].values

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0])

In [10]:
target.iloc[17]

0

In [11]:
depth_to_check = [4,5,6,7,8,9,10,15,20,30,50,100]

for i in depth_to_check:
    dt, node_count, max_depth, train_test_full_error  = dec_tree_df(X_train,y_train, X_test, y_test, i, 42)
    
    name_file = f"{PATH_MODELS}\\reverse_model1\\{SIZE}\\decision_tree_size{SIZE}_reverse_df_model_{str(i)}.pkl"
    with open(name_file, 'wb') as file:
        pickle.dump(dt, file)
    
    print(node_count, max_depth)
    print(train_test_full_error)

31 4
              train      test
accuracy   0.665112  0.665214
precision  0.557704  0.569876
recall     0.358407  0.367306
f1         0.436377  0.446698
63 5
              train      test
accuracy   0.669619  0.666933
precision  0.575738  0.583090
recall     0.329261  0.332444
f1         0.418935  0.423457
127 6
              train      test
accuracy   0.671092  0.668528
precision  0.569529  0.576232
recall     0.371472  0.374479
f1         0.449658  0.453948
255 7
              train      test
accuracy   0.673520  0.671536
precision  0.579100  0.586494
recall     0.356579  0.363636
f1         0.441380  0.448929
511 8
              train      test
accuracy   0.678444  0.674911
precision  0.592696  0.595774
recall     0.354938  0.362135
f1         0.443991  0.450462
1023 9
              train      test
accuracy   0.683033  0.671781
precision  0.604470  0.590338
recall     0.357898  0.352627
f1         0.449597  0.441520
2033 10
              train      test
accuracy   0.688332  0.6739

In [12]:
amount_features = len(reverse_df_sort.columns) - SIZE*SIZE #the previous boards
features = reverse_df_sort.iloc[:, :amount_features]
for i in range(SIZE*SIZE): # to any pixel in the expected board
    name_col = 'Col_' + str(i+amount_features + 1)
    target = reverse_df_sort[name_col]
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.1, random_state=613)
    dt, node_count, max_depth, train_test_full_error  = dec_tree_df(X_train,y_train, X_test, y_test, 10, 42)
    name_file = f"{PATH_MODELS}\\reverse1\\{SIZE}\\dt_size{SIZE}_r1_pixel_{str(i+1)}.pkl"
    with open(name_file, 'wb') as file:
        pickle.dump(dt, file)
    print(i)
    print(node_count, max_depth)
    print(train_test_full_error)

0
2031 10
              train      test
accuracy   0.689000  0.670185
precision  0.616587  0.571030
recall     0.375543  0.347782
f1         0.466784  0.432284
1
2035 10
              train      test
accuracy   0.690316  0.673684
precision  0.614670  0.576717
recall     0.391010  0.365087
f1         0.477969  0.447125
2
2043 10
              train      test
accuracy   0.690439  0.669510
precision  0.612138  0.576152
recall     0.393450  0.370687
f1         0.479015  0.451126
3
2035 10
              train      test
accuracy   0.690882  0.674972
precision  0.614039  0.580510
recall     0.402315  0.374006
f1         0.486124  0.454920
4
2039 10
              train      test
accuracy   0.690316  0.679698
precision  0.620256  0.584611
recall     0.374412  0.365770
f1         0.466952  0.449995
5
2033 10
              train      test
accuracy   0.689566  0.674359
precision  0.611316  0.578154
recall     0.393809  0.365042
f1         0.479029  0.447522
6
2037 10
              train      test


In [13]:
X_test

,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8,Col_9,Col_10,...,Col_16,Col_17,Col_18,Col_19,Col_20,Col_21,Col_22,Col_23,Col_24,Col_25
96613,0,1,1,0,0,0,0,0,1,0,...,0,0,0,1,0,1,1,1,0,0
6315,0,0,0,0,0,0,0,0,0,0,...,1,1,0,1,0,1,1,0,0,0
93520,0,1,0,1,1,0,0,1,1,0,...,0,1,0,1,0,1,0,0,1,1
156445,1,1,1,0,0,1,1,1,0,0,...,0,0,1,1,0,0,1,0,0,0
41032,0,0,0,0,1,1,1,0,1,0,...,0,0,0,0,0,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128791,1,0,0,1,1,1,0,1,0,0,...,1,0,0,1,0,0,0,0,0,0
22484,0,0,0,0,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
157898,1,1,1,0,1,1,0,1,0,0,...,1,0,1,0,1,1,0,1,0,1
75482,0,0,1,1,1,0,1,1,0,0,...,0,0,0,1,0,0,0,1,1,1


In [14]:
PATH_DATA_TEST = 'C:\\GameOfLifeFiles\\dataTest\\'

In [15]:
name_file = f"{PATH_DATA_TEST}\\data_test_size_{SIZE}.pkl"
with open(name_file, 'wb') as file:
    pickle.dump(X_test, file)